Creating 3 boosted tress (via 3 fold cross validation). Sum the probabilities and pick the top 5.
The hyper parameters were chosen based on a 1% of the data.

In [1]:
import pandas as pd
import numpy as np
import graphlab as gl
import cPickle as pkl
import operator
#from sklearn.externals import joblib

In [2]:
cols = ['site_name', u'posa_continent','user_location_country', u'user_location_region',
         u'user_location_city', u'is_mobile', u'is_package', u'channel', 
         u'srch_destination_id','hotel_cluster', u'srch_destination_type_id', u'B_Weekday',
         u'B_Hour',  u'C_weekofYear', u'adv_days', u'duration',
         u'srch_adults_cnt', u'srch_children_cnt', u'srch_rm_cnt']

In [3]:
# Transform the dates
def transform_dates(data):    
    data['B_Hour'] = data['date_time'].dt.hour
    data['B_Weekday'] = data['date_time'].dt.weekday
    data['C_weekofYear'] = data['srch_ci'].dt.day
    data['duration'] = (data['srch_co'] - data['srch_ci']).dt.days
    # Remove time stamp but retain the dates
    data['date_time'] = data['date_time'].dt.normalize()
    data['adv_days'] = (data['srch_ci'] - data['date_time']).dt.days
    
    return data

In [4]:
# Drop columns
def drop_Cols(data):
    drop_columns = ['date_time','user_id', 'srch_ci', 'srch_co']
    if 'is_booking' in data.columns:
        drop_columns.append('is_booking')
    if 'cnt' in data.columns:
        drop_columns.append('cnt')
    
    data = data.drop(drop_columns,axis=1)
    
    return data

In [5]:
def reorderCols(data):
    # ReOrder Columns
    new_order = cols
    if 'hotel_cluster' not in data.columns:
        new_order.remove('hotel_cluster')
    data = data[new_order]
    
    return data

In [7]:
col_type = {}
for i,col in enumerate(cols):
    if i < 14:
        col_type[col] = 'str'

In [8]:
data_booked = pd.read_csv('booked.csv',dtype=col_type,parse_dates=['date_time','srch_ci','srch_co'],
                          infer_datetime_format=True)

In [9]:
data_booked = reorderCols(drop_Cols(transform_dates(data_booked)))
data_booked[['B_Weekday','B_Hour','C_weekofYear']] = data_booked[['B_Weekday','B_Hour','C_weekofYear']].astype(str)

In [10]:
data_test = pd.read_csv('original_data/test.csv',dtype=col_type,parse_dates=['date_time','srch_ci','srch_co'],
                          infer_datetime_format=True)
# Fix Errors
data_test.srch_ci = pd.to_datetime(data_test.srch_ci,errors='coerce')
ind = data_test.srch_ci[data_test.srch_ci.isnull()==True].index
# Replace with most common date
data_test.srch_ci.ix[ind] = data_test.srch_ci.describe()['top'].date()

# And with srch_co
ind = data_test.srch_co[data_test.srch_co.isnull()==True].index
data_test.srch_co.ix[ind] = data_test.srch_co.describe()['top'].date()

C:\Users\Shetty\Anaconda\lib\site-packages\pandas\core\indexing.py:117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [11]:
data_test = reorderCols(drop_Cols(transform_dates(data_test)))
data_test[['B_Weekday','B_Hour','C_weekofYear']] = data_test[['B_Weekday','B_Hour','C_weekofYear']].astype(str)

In [12]:
with open('parma_best.pickle','rb') as f:
    params_best = pkl.load(f)

In [13]:
sf_full_train = gl.SFrame(data_booked)
del data_booked
sf_full_test = gl.SFrame(data_test)
del data_test

[INFO] graphlab.cython.cy_server: GraphLab Create v1.10.1 started. Logging: C:\Users\Shetty\AppData\Local\Temp\graphlab_server_1465587779.log.0
INFO:graphlab.cython.cy_server:GraphLab Create v1.10.1 started. Logging: C:\Users\Shetty\AppData\Local\Temp\graphlab_server_1465587779.log.0


This non-commercial license of GraphLab Create is assigned to vedshetty@yahoo.com and will expire on November 17, 2016. For commercial licensing options, visit https://dato.com/buy/.


In [14]:
kFold_split = gl.cross_validation.KFold(sf_full_train,3)

In [15]:
for i,(train,_) in enumerate(kFold_split):
    model = gl.boosted_trees_classifier.create(train,validation_set=None,**params_best)
    sf_predict = model.predict_topk(sf_full_test,k=5)
    fname = 'gl_boost_' + str(i)
    sf_predict.save(fname)

Boosted trees classifier:

--------------------------------------------------------

Number of examples          : 2000459

Number of classes           : 100

Number of feature columns   : 18

...

| 7         | 1420.102532  | 0.126721          | 3.848890          |

| 8         | 1617.480106  | 0.130157          | 3.825990          |

| 9         | 1823.441104  | 0.132961          | 3.806735          |

| 10        | 2028.616605  | 0.135653          | 3.790183          |

+-----------+--------------+-------------------+-------------------+

In [18]:
data1 = gl.load_sframe('gl_boost_0/')
data2 = gl.load_sframe('gl_boost_1/')
data3 = gl.load_sframe('gl_boost_2/')

In [28]:
print data1.head(),data2.head(),data3.head()

+----+-------+-----------------+
| id | class |   probability   |
+----+-------+-----------------+
| 0  |   44  | 0.0272819083184 |
| 0  |   93  |  0.026366751641 |
| 0  |   90  | 0.0244428925216 |
| 0  |   67  | 0.0243551433086 |
| 0  |   22  | 0.0242601949722 |
| 1  |   93  | 0.0240532513708 |
| 1  |   90  | 0.0204251669347 |
| 1  |   86  | 0.0195579454303 |
| 1  |   92  | 0.0192906558514 |
| 1  |   73  | 0.0192765835673 |
+----+-------+-----------------+
[10 rows x 3 columns]
 +----+-------+-----------------+
| id | class |   probability   |
+----+-------+-----------------+
| 0  |   58  | 0.0275340694934 |
| 0  |   90  | 0.0262247417122 |
| 0  |   22  | 0.0235852543265 |
| 0  |   44  | 0.0235495232046 |
| 0  |   86  | 0.0215876009315 |
| 1  |   90  | 0.0218421667814 |
| 1  |   93  | 0.0216993335634 |
| 1  |   92  | 0.0193369742483 |
| 1  |   86  |  0.01916542463  |
| 1  |   58  | 0.0178132709116 |
+----+-------+-----------------+
[10 rows x 3 columns]
 +----+-------+----------------

In [26]:
data1 = data1.sort([('id',True),('probability',False)])
data2 = data2.sort([('id',True),('probability',False)])
data3 = data3.sort([('id',True),('probability',False)])

In [27]:
data1.save('gl_boost_0/')
data2.save('gl_boost_1/')
data3.save('gl_boost_2/')

In [170]:
# Append data from all three predictions
data1_3 = data1.append(data2.append(data3))

In [171]:
# Sum the probabilites by id and class
data1_3 = data1_3.groupby(['id','class'],operations={'Agg':gl.aggregate.SUM('probability')})

In [173]:
# Create a dict for each id.
data1_4 = data1_3.groupby('id',operations={'class_prob':gl.aggregate.CONCAT('class', 'Agg')})

In [192]:
data1_4.head()

id,class_prob
0,"{'58':0.02753406949341297, ..."
1,"{'58':0.017813270911574364, ..."
2,"{'23':0.0766739659011364, ' ..."
3,"{'1':0.33526166528463364, ..."
4,"{'91':0.07791242934763432, ..."
5,"{'39':0.06655545718967915, ..."
6,"{'39':0.024916259571909904, ..."
7,"{'39':0.021381346508860588, ..."
8,"{'24':0.3784366548061371, ' ..."
9,"{'39':0.01657235436141491, ..."


In [189]:
data1_4 = data1_4.sort('id')

In [193]:
def pickTop5(predictions):
    predictions = predictions['class_prob']
    top5 = sorted(predictions.items(),key=operator.itemgetter(1),reverse=True)[:5]
    return map(lambda x: x[0],top5)
    

In [194]:
top5 = np.array(data1_4.apply(pickTop5))

In [223]:
def savePred2(pred_array):
    df = pd.DataFrame(columns=['hotel_cluster'],index=np.arange(pred_array.shape[0]))
    df['hotel_cluster'] = map(lambda x: ' '.join(x),pred_array)
    df.to_csv('submission_2.csv',index=True,index_label='id')

In [226]:
savePred2(top5)